In [1]:
'''

This file has the following goals:
""
- process downloaded html, and pick one cannonical string fro each url (out of
all pages of html downloaded, or set to empty if no valid html was fetched)
- 

FORGOT TO COMBINE WITH PAST urls with texts...

'''

'\n\nThis file has the following goals:\n\n- process downloaded html, and pick one cannonical string fro each url (out of\nall pages of html downloaded, or set to empty if no valid html was fetched)\n- \n\n\n'

In [1]:
from newspaper import Article
import json
import time
from tqdm import tqdm
from joblib import Parallel, delayed



In [2]:
# this simply extract the relevant news text from the given html
# url is completely optional (should not affect the code -- I think!)
def process_html(html, url='a.com'):

    
    article = Article(url)

    article.download(input_html=html)
    article.parse()
    return article.text



In [17]:
'''

Load the downloaded commoncrawl (downloaded using comcrawl package) 
and the filtered edit data (from Testing_comcrawl)

'''

# load the counterfactual grounding edit pairs
data = []
with open('C://Users/t-pewest/Desktop/Data/processing_3/dataset_small.json') as f:
    for line in f:
        data += [json.loads(line)]
        
        
# load the downloaded webpages
with open('C://Users/t-pewest/Desktop/Data/processing_3/all.json') as f:
    comcrawl_dump = json.loads(f.read())
    
    
print(len(comcrawl_dump))
# load the downloaded webpages
with open('C://Users/t-pewest/Desktop/Code/final_html_out.json') as f:
    comcrawl_dump_old = json.loads(f.read())
    
# include what we had from the old dump (the new dump only grabbed novel pages)
comcrawl_dump.update(comcrawl_dump_old)

print(len(comcrawl_dump))




25476
48271


In [18]:
'''

This block processes the comcrawl dump, getting the text when available for html (do it in
parallel to make it faster...)

'''

def process_url(inp):
    i, url, article = inp
    print(i)
    texts = []
    for v in article:
        try:
            texts += [process_html(v['html'])]
        except:
            pass
    
    return (i, url, texts)
    


processed_outputs = Parallel(n_jobs=16)(delayed(process_url)(inp) for inp in [(i, url, comcrawl_dump[url]) for (i, url) in enumerate(comcrawl_dump.keys())])


In [19]:
# put it back in a dictionary format
urls_with_texts = {v[1]:v[2] for v in processed_outputs}
    
    
with open('C://Users/t-pewest/Desktop/Data/processing_3/urls_with_texts.json', 'w') as f:
    json.dump(urls_with_texts, f)

In [20]:
'''

This block assigns one of the processed texts to each of the urls (in processed_urls,
we have 0 to many options, so we pick one of these)

'''



# remove repeats
for url in urls_with_texts.keys():
    urls_with_texts[url] = list(set(urls_with_texts[url]))
    

    

#
# Section below makes a dictionary of urls with nonzero texts
#
domains =['.com', '.uk','.org','.ca', '.au','.nz','.net']


def check_no_extension(url):
    return any([url.endswith(d) for d in domains])

def valid_url(url):
    
    return not check_no_extension(url) and 'youtube.com' not in url


# function to get longest text...
get_longest_text = lambda v: max(v, key=len)
    
urls_with_nonzero_texts = {}
for url in urls_with_texts.keys():
    if valid_url(url) and len(urls_with_texts[url]) > 0:
        urls_with_nonzero_texts[url] = get_longest_text(urls_with_texts[url])
    
print(len(urls_with_nonzero_texts))

7546


In [21]:

for d in data:
    d['article_text'] = {'edit':[], 'orig':[]}
    
    for url in [u for u in d['urls_edit'] if u not in d['urls_orig']]:
        if url in urls_with_nonzero_texts:
            d['article_text']['edit'] += [{'url':url, 'text':urls_with_nonzero_texts[url]}]
    

    top_url_edit = '' if len(d['article_text']['edit']) == 0 else d['article_text']['edit'][0]['url']
    
    for url in [u for u in d['urls_orig']]:
        if url in urls_with_nonzero_texts:
            d['article_text']['orig'] += [{'url':url, 'text':urls_with_nonzero_texts[url]}]   


In [22]:
counterfactual_grounding = [d for d in data if len(d['article_text']['edit'])>0 and len(d['article_text']['orig'])>0]
print(len(counterfactual_grounding))

# clean this up so that it doesn't have all the extra information
counterfactual_grounding_clean = [{'context': d['prefix'],
                                 'grounding_0': d['article_text']['edit'][0]['text'],
                                 'target_0':d['edited'],
                                 'grounding_1':d['article_text']['orig'][0]['text'],
                                 'target_1':d['original'] } for d in counterfactual_grounding]


487


In [23]:
for d in counterfactual_grounding_clean[10:20]:
    print('\n\n\n========================')
    for key in ['context','target_0', 'target_1']:
        print(key + '===\n')
        print(d[key])




context===

Major wars<!-- linked from redirect "Major wars"--> 1,000+ deaths per year Conflicts in the following list are currently causing at least 1000 violent deaths per year, a categorization used by the Uppsala Conflict Data Program  Start of conflict Conflict Continent Location Cumulative fatalities Fatalities in 2013 Fatalities in 2014 1978 War in Afghanistan Asia Afghanistan 1405111 1,405,111-2,084,468See  -1,000+ Other conflicts - Fewer than 1,000 deaths per year Note: Listed conflicts are at least 25 cumulative deaths in total and at least 1 death in current or past year. Italics indicate disputed territories and unrecognized states. class="wikitable sortable" Start of conflict Conflict Continent Location data-sort-type="number" | Cumulative fatalities data-sort-type="number" | Fatalities in 2013 data-sort-type="number" | Fatalities in 2014 align=center | 1918 Kurdish separatism in Iran Asia Iran 34000 34,000+  38 38-39a a 3 3+ |align=center | 1945 Korean conflict  Asia N

In [24]:
## need to dump the combined html pile for future versions of the dataset
with open('C://Users/t-pewest/Desktop/Code/comcraw_db_2.json','w') as f:
    f.write(json.dumps(comcrawl_dump))

In [25]:
counterfactual_target = [d for d in data if len(d['article_text']['edit'])>0 or len(d['article_text']['orig'])>0]

In [51]:
counterfactual_target[4]

{'prefix': 'Notably, Martin\'s work makes a sharp departure from the prevalent "heroic knights and chivalry" schema that has become a mainstay in fantasy as derived from J.R.R. Tolkien\'s The Lord of the Rings. He specifically critiques the oversimplification of Tolkien\'s themes and devices by imitators in ways that he has humorously described as "Disneyland Middle Ages", The author makes a point of grounding his work on a foundation of historical fiction, which he channels to evoke important social and political elements of primarily the European medieval era that differ markedly from elements of modern times, including the multigenerational, rigid, and often brutally consequential nature of the hierarchical class system of feudal societies',
 'original': 'Relationship with fans Blog right|thumb|Martin signing books in a bookstore in Ljubljana, Slovenia (June 2011)Martin actively contributes to his blog, Not a Blog; in April 2018 he moved his blog from Livejournal to his own website.

In [45]:
comcrawl_dump['https://www.washingtonpost.com/news/volokh-conspiracy/wp/2014/08/01/does-israel-have-to-give-free-power-to-gaza/']

[]

In [47]:
with open('t1.txt',encoding='utf8') as f:
    t1 = ''
    for line in f.readlines():
        t1 += line
        
with open('t2.txt',encoding='utf8') as f:
    t2 = ''
    for line in f.readlines():
        t2 += line

'<!DOCTYPE html><html lang="en"><head><title>Does Israel have to give free power to Gaza? - The Washington Post</title><meta name="description" content=""/><meta property="og:site_name" content="Washington Post"/><meta property="og:type" content="article"/><meta property="og:url" content="https://www.washingtonpost.com/news/volokh-conspiracy/wp/2014/08/01/does-israel-have-to-give-free-power-to-gaza/"/><meta property="og:image" content="https://www.washingtonpost.com/resizer/2CjPNwqvXHPS_2RpuRTKY-p3eVo=/1484x0/www.washingtonpost.com/pb/resources/img/twp-social-share.png"/><meta property="og:title" content="Does Israel have to give free power to Gaza?"/><meta property="og:description" content=""/><meta property="twitter:site" content="@WashingtonPost"/><meta property="twitter:card" content="summary_large_image"/><meta property="twitter:title" content="Does Israel have to give free power to Gaza?"/><meta property="twitter:description" content=""/><meta name="last_updated_date" content="20

In [49]:
process_html(t1)

'His memo drew an unusual response from Prof. David Enoch of Hebrew University, who not only disagreed with Bell, but through the good offices of Brian Leiter, called for the legal academy to impose some kind of reputational sanctions against Bell. I may deal with the “academic bullying” aspect in a latter post. Here I will address the substance. I hope my views of the merits are not colored by the prospect of facing the kind of academic ostracism threatened by Prof. Enoch.\n\nCivilian power stations are legitimate military targets in an armed conflict, and have been heavily targeted by Western countries in most recent conflicts. Thus even if Israel deliberately targeted the Gaza power plant, this would be well within international practice. Yet the dispute is not about bombing civilian power facilities – a subject much discussed in law of war manuals and treatises – but rather the more particular claim that Israel cannot switch off the power it provides to Gaza from its own power stat

In [50]:
process_html(t2)

'A recent debate has arisen over whether Israel, from which Gaza procures much of its power, is obligated under international law to continue providing Gaza with power during the hostilities between them. Prof. Avi Bell of San Diego and Bar Ilan Universities, and my colleague at the Kohelet Policy Forum, wrote a detailed analysis of the question, and concluded there is no requirement to provide electricity.\n\nHis memo drew an unusual response from Prof. David Enoch of Hebrew University, who not only disagreed with Bell, but through the good offices of Brian Leiter, called for the legal academy to impose some kind of reputational sanctions against Bell. I may deal with the “academic bullying” aspect in a latter post. Here I will address the substance. I hope my views of the merits are not colored by the prospect of facing the kind of academic ostracism threatened by Prof. Enoch.\n\nCivilian power stations are legitimate military targets in an armed conflict, and have been heavily targe